In [ ]:
# Importing Built Tools such as Notion , Slack
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END , START
# from langgraph.checkpoint import MemorySaver
from IPython.display import Image, display
from typing import Optional , Literal , TypedDict
from datetime import datetime
from langgraph.prebuilt import ToolNode
from langchain.tools import tool
import re
import os
from dotenv import load_dotenv


from notion_api_tools import  append_toggle_with_bullets_for_change_log , update_block_content , delete_block , append_bulleted_list_to_block
from slack_tools import retrieve_channel_id_by_name , post_message_to_channel , fetch_replies_from_channel



class MyAgentState1(TypedDict):
    latest_meeting_topic : Optional[str]
    latest_email_meeting_summary : Optional[str]
    latest_action_items_data : Optional[str]
    structured_refined_email_summary : Optional[dict]
    structured_list_of_action_items : Optional[list]
    notion_page_ids_list: Optional[list[dict]]
    slack_channel_id : Optional[str]
    slack_bot_id : Optional[str]
    pre_existing_notion_page_data: Optional[list[dict]]
    suggested_structured_changes: Optional[list]
    next_node: Optional[str]  # Add this to track the next node
    new_page_content_list : Optional[list]


In [3]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
# print(openai_api_key)

In [ ]:

from notion_api_tools import add_page_to_action_items_database_table_by_id , get_each_notion_page_action_items_table_id_mapping

@tool
def process_changes(data_list):
    """
    Processes a list of changes for Notion pages by updating, deleting, or appending content,
    and logs changes and suggested action items.

    Args:
        data_list (list): A list of dictionaries where each dictionary contains:
            - "page_id" (str): The Notion page ID.
            - "changes" (list): A list of change dictionaries containing:
                - "objectId" (str): The block ID to be modified.
                - "ChangeType" (str): The type of change ("update", "delete", "append").
                - "ContentForChange" (str): The new content for updates or append actions.
            - "changeLogs" (list): A list of change log messages.
            - "suggested_action_items_add" (list): A list of suggested action item dictionaries containing:
                - "action_item_text" (str): The action item description.
                - "status" (str): The current status of the action item.
                - "assignees" (list): List of assigned persons.
    
    Returns:
        None. The function applies the requested changes directly to the Notion page.
    """
    today_date = datetime.today().strftime('%Y-%m-%d')
    notion_page_to_action_items_table_id_matching = get_each_notion_page_action_items_table_id_mapping()
    
    for entry in data_list:
        page_id = entry.get("page_id")
        changes = entry.get("changes", [])
        change_logs = entry.get("changeLogs", [])
        suggested_action_items_list = entry.get("suggested_action_items_add", [])
        
        for change in changes:
            block_id = change.get("objectId")
            change_type = change.get("ChangeType")
            content = change.get("ContentForChange")
            
            if change_type == "update":
                update_block_content.invoke(input={"updating_block_info": {"blockId": block_id, "new_text_content": content}})
            elif change_type == "delete":
                delete_block.invoke(input={"deleting_block_info": {"blockId": block_id}})
            elif change_type == "append":
                append_bulleted_list_to_block.invoke(input={"adding_content_info": {"blockId": block_id, "bullet_points_list": [content]}})
        
        if change_logs:
            append_toggle_with_bullets_for_change_log.invoke(input={"addingToggleItemInfo": {
                "NOTION_PAGE_ID": page_id,
                "toggle_item_text": f"Change Log {today_date}",
                "bullet_points_list": change_logs
            }})
        
        if suggested_action_items_list:
            for each_action_item_info in suggested_action_items_list:
                temp_dict = {
                    "action_item": each_action_item_info.get("action_item_text"),
                    "status": each_action_item_info.get("status"),
                    "assigned_to": each_action_item_info.get("assignees")
                }
                add_page_to_action_items_database_table_by_id(
                    notion_page_to_action_items_table_id_matching.get(page_id), temp_dict
                )

In [ ]:
from notion_api_tools import add_heading_to_page , add_bulleted_list_with_subpoints , append_new_topic_toggle_under_given_toggle_id , append_toggle_to_given_page , append_toggle_with_bullets_for_change_log , create_notion_table , add_new_notion_page_data_to_existing_notion_pages_database , get_each_notion_page_action_items_table_id_mapping , add_each_notion_page_action_items_table_id_mapping
from datetime import datetime

@tool
def applying_new_topics_in_new_notion_page(notion_page_id, latest_meeting_topic, latest_notes_list, extracted_action_items_list):
    """
    Applies new topics and notes to a new Notion page, organizes them into toggles, and updates action items.
    
    Args:
        notion_page_id (str): The Notion page ID where the topics and notes will be added.
        latest_meeting_topic (str): The main topic of the latest meeting.
        latest_notes_list (list): A list of dictionaries where each dictionary contains:
            - "topic" (str): The main topic name.
            - "sub_topics" (list): A list of sub-topic dictionaries, where each dictionary contains:
                - "sub_topic" (str): The sub-topic name.
                - "bullet_points" (list): A list of bullet point strings related to the sub-topic.
        extracted_action_items_list (list): A list of dictionaries where each dictionary contains:
            - "action_item" (str): The description of the action item.
            - "status" (str): The current status of the action item.
            - "assigned_to" (list): A list of assignees for the action item.
    
    Returns:
        None. The function updates the Notion page with the new topics, notes, and action items.
    """
    today_date = datetime.today().strftime('%Y-%m-%d')
    
    # Step 1: Applying Heading
    add_heading_to_page(notion_page_id, "This is related to Change Log")
    
    # Step 2: Adding Toggle with Change Log
    append_toggle_with_bullets_for_change_log.invoke(input={"addingToggleItemInfo": {
        "NOTION_PAGE_ID": notion_page_id,
        "toggle_item_text": f"Change Log {today_date}",
        "bullet_points_list": ["Added all new Topics"]
    }})
    
    # Step 3: Adding Latest Notes Section
    latest_notes_toggle_block_id = append_toggle_to_given_page(notion_page_id, "Latest Notes")
    
    # Appending all topic blocks as children to latest_notes_toggle_block_id
    for each_topic_element in latest_notes_list:
        topic_name = each_topic_element.get("topic")
        sub_topics_list = each_topic_element.get("sub_topics")  # List of sub-topic dictionaries
        
        topic_toggle_block_id = append_new_topic_toggle_under_given_toggle_id(latest_notes_toggle_block_id, topic_name)
        
        # Appending each sub-topic with corresponding bullet points
        add_bulleted_list_with_subpoints(topic_toggle_block_id, sub_topics_list)
    
    # Step 4: Adding Action Items Table
    created_table_id = create_notion_table(notion_page_id)
    print("At creation of Action Items table")
    
    for each_action_item in extracted_action_items_list:
        each_action_item_info = {
            "action_item": each_action_item.get("action_item_text"),
            "status": each_action_item.get("status"),
            "assigned_to": each_action_item.get("assignees")
        }
        add_page_to_action_items_database_table_by_id(created_table_id, each_action_item_info)
    
    print("Adding content to new Notion Page completed, storing details of newly created Notion Page")
    
    # Adding newly created Notion Page Info
    add_new_notion_page_data_to_existing_notion_pages_database(notion_page_id, latest_meeting_topic)
    
    print("Trying to add New Notion Page ID mapping to created Action Items Table")
    add_each_notion_page_action_items_table_id_mapping(notion_page_id, created_table_id)

In [ ]:
from slack_tools import handle_sending_msg
from notion_api_tools import validate_notion_page

#checking whether notion page id is valid or not
async def get_valid_notion_page(state):
    
    while True:
        try:
            human_response = await handle_sending_msg(state["latest_meeting_topic"])
            print(f"📩 Received Notion Page ID: {human_response}")

            if human_response and await validate_notion_page(human_response):
                print("✅ Notion Page ID is valid!")
                return human_response
            else:
                print("❌ Notion Page ID is invalid. Asking admin again...")
                await handle_sending_msg("The Notion Page ID is invalid. Please provide a correct one.")

        except Exception as e:
            print(f"❌ Error while validating Notion Page: {e}")

In [ ]:
from notion_api_tools import fetch_data_from_meetings_history_database_table ,fetch_data_from_notion_pages_data_database_table, fetch_notion_page_content , append_bulleted_list_to_block
import json
import re
import time
from notion_api_tools import update_block_content , delete_block , append_toggle_with_bullets_for_change_log  # ,  append_bulleted_list_to_block
from datetime import datetime
from slack_tools import handle_sending_msg

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

In [ ]:
def getting_meeting_from_email(state :MyAgentState1)-> MyAgentState1 :

    print("Here we will get both Topic and Latest Email Summary as Input from Agent Invoking")
    print("Invoked getting_meeting_from_email")
    print("Getting latest Meeting to Invoke Agent")
    # state["latest_meeting_topic"] = "Hive Reviews and Demos"
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print(f"Latest Meeting topic got is {state["latest_meeting_topic"]}")
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("LATEST MEETING SUMMARY IS \n")
    print(state["latest_email_meeting_summary"])
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("\nLATEST ACTION ITEMS DATA\n")
    print(state["latest_action_items_data"])
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

    print("Refining Summary \n")
    meeting_topic = state["latest_meeting_topic"]
    latest_meeting_summary = state["latest_email_meeting_summary"]



    prompt = f"""
        Given the following call summary:
        1. Identify the **title** of this meeting.
        2. Remove any **duplicated sentences** while keeping the content meaningful.
        3. Identify the **stakeholders** in the call:
        - Who is **leading the call**?
        - Who has been **assigned tasks**?
        4. Identify the **action points** in the call and create a table of users and their assigned tasks.

        Call Summary:
        {latest_meeting_summary}  # Access as dictionary key

        Provide the output **strictly in JSON format**, without any extra text.

        Example:
        {{
            "meeting_title": {meeting_topic},
            "cleaned_summary": "A cleaned version of given summary without any odd words",
            "stakeholders": {{
                "leader": "John Doe",
                "task_assignees": ["Alice", "Bob"]
            }},
            "action_items": [
                {{"user": "Alice", "task": "Prepare budget report"}},
                {{"user": "Bob", "task": "Review funding options"}}
            ]
        }}
        """
    
    refined_structured_summary = llm.invoke(prompt)
    refined_llm_content = refined_structured_summary.content
    print("Checking Completed")
    # print(llm_content)
    # changing llm_content to json and extracting is_new_topic 
    cleaned_json_str = re.sub(r'^```json\n|\n```$', '', refined_llm_content.strip())
    try:
        json_data = json.loads(cleaned_json_str)
        print("Refined Email Summary \n")
        print(json_data)
        state["structured_refined_email_summary"] = json_data
    except Exception as e:
        print(e)

    return state

In [ ]:
# defining Tools Node here 
tools = [process_changes , applying_new_topics_in_new_notion_page]

In [ ]:
def checking_new_or_old_topic(state : MyAgentState1) -> MyAgentState1 :
    print("Invoked  checking_new_or_old_topic")
    fetched_past_meetings_info = fetch_data_from_meetings_history_database_table("197e35223beb80039714f0cd468bce2e")
    latest_meeting_topic = state["latest_meeting_topic"]
    latest_meeting_info = {
        "meeting_name" : latest_meeting_topic,
        "happened_date" : "February 11, 2025"
    }
    
